In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#https://towardsdatascience.com/your-first-neural-network-in-pytorch-725631ae0fc

In [12]:
iris = pd.read_csv('https://raw.githubusercontent.com/pandas-dev/pandas/master/pandas/tests/data/iris.csv')
iris.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [13]:
iris.describe()

,SepalLength,SepalWidth,PetalLength,PetalWidth
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [15]:
iris.groupby('Name').describe()

SepalLength                                              \
                      count   mean       std  min    25%  50%  75%  max   
Name                                                                      
Iris-setosa            50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8   
Iris-versicolor        50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0   
Iris-virginica         50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9   

                SepalWidth         ... PetalLength      PetalWidth         \
                     count   mean  ...         75%  max      count   mean   
Name                               ...                                      
Iris-setosa           50.0  3.418  ...       1.575  1.9       50.0  0.244   
Iris-versicolor       50.0  2.770  ...       4.600  5.1       50.0  1.326   
Iris-virginica        50.0  2.974  ...       5.875  6.9       50.0  2.026   

                                                    
                      std  min  25%  50%  75%  max  
Name                                                
Iris-setosa      0.107210  0.1  0.2  0.2  0.3  0.6  
Iris-versicolor  0.197753  1.0  1.2  1.3  1.5  1.8  
Iris-virginica   0.274650  1.4  1.8  2.0  2.3  2.5  

[3 rows x 32 columns]

In [16]:
mappings = {
   'Iris-setosa': 0,
   'Iris-versicolor': 1,
   'Iris-virginica': 2
}
iris['Name'] = iris['Name'].apply(lambda x: mappings[x])

In [17]:
iris.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [18]:
iris.groupby('Name').describe()

SepalLength                                             SepalWidth  \
           count   mean       std  min    25%  50%  75%  max      count   
Name                                                                      
0           50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8       50.0   
1           50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0       50.0   
2           50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9       50.0   

             ... PetalLength      PetalWidth                                  \
       mean  ...         75%  max      count   mean       std  min  25%  50%   
Name         ...                                                               
0     3.418  ...       1.575  1.9       50.0  0.244  0.107210  0.1  0.2  0.2   
1     2.770  ...       4.600  5.1       50.0  1.326  0.197753  1.0  1.2  1.3   
2     2.974  ...       5.875  6.9       50.0  2.026  0.274650  1.4  1.8  2.0   

                
      75%  max  
Name            
0     0.3  0.6  
1     1.5  1.8  
2     2.3  2.5  

[3 rows x 32 columns]

In [19]:
X = iris.drop('Name', axis=1).values
y = iris['Name'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [22]:
X_train[:3]

tensor([[4.6000, 3.6000, 1.0000, 0.2000],
        [5.7000, 4.4000, 1.5000, 0.4000],
        [6.7000, 3.1000, 4.4000, 1.4000]])

In [23]:
y_train[:3]

tensor([0, 0, 1])

In [33]:
class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=4, out_features=16)
        self.fc2 = nn.Linear(in_features=16, out_features=12)
        self.output = nn.Linear(in_features=12, out_features=3)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.output(x)
        return x

In [34]:
model = ANN()
model

ANN(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=12, bias=True)
  (output): Linear(in_features=12, out_features=3, bias=True)
)

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [36]:
%%time
epochs = 100
loss_arr = []
for i in range(epochs):
   y_hat = model.forward(X_train)
   loss = criterion(y_hat, y_train)
   loss_arr.append(loss)
 
   if i % 10 == 0:
       print(f'Epoch: {i} Loss: {loss}')
 
   optimizer.zero_grad()
   loss.backward()
   optimizer.step()

Epoch: 0 Loss: 1.1083999872207642
Epoch: 10 Loss: 0.8317091464996338
Epoch: 20 Loss: 0.47312065958976746
Epoch: 30 Loss: 0.29335421323776245
Epoch: 40 Loss: 0.15963959693908691
Epoch: 50 Loss: 0.09282132238149643
Epoch: 60 Loss: 0.07100081443786621
Epoch: 70 Loss: 0.06352565437555313
Epoch: 80 Loss: 0.06024404242634773
Epoch: 90 Loss: 0.05841391161084175
CPU times: user 123 ms, sys: 9.41 ms, total: 132 ms
Wall time: 288 ms


In [37]:
preds = []
with torch.no_grad():
   for val in X_test:
       y_hat = model.forward(val)
       preds.append(y_hat.argmax().item())

In [40]:
df = pd.DataFrame({'Y': y_test, 'YHat': preds})
df['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]
df.head()

,Y,YHat,Correct
0,1,1,1
1,0,0,1
2,2,2,1
3,1,1,1
4,1,1,1


In [39]:
df['Correct'].sum() / len(df)

1.0

In [47]:
for coeff in model.parameters():
    print(coeff)

Parameter containing:
tensor([[ 0.5995,  0.9221,  0.0185, -0.1045],
        [ 0.8786,  0.2146, -0.8884, -0.6937],
        [-0.2673, -0.3511,  0.4017,  0.0943],
        [ 0.7528,  0.6937, -0.4179, -0.9780],
        [-0.1641, -0.2276,  0.8625,  1.0309],
        [ 0.0738, -0.3253,  0.5694,  0.9658],
        [ 0.0432, -0.4276, -0.1959, -0.4076],
        [ 0.0014, -0.3402,  0.0492, -0.4403],
        [ 0.6641,  0.0311,  0.4729, -0.0538],
        [-0.2154, -0.1274, -0.2839,  0.4080],
        [-0.3138,  0.1974,  0.0214, -0.4698],
        [ 0.2462, -0.6930,  0.4893,  0.2996],
        [ 0.1210, -0.2522,  0.8155,  0.3603],
        [-0.0489, -0.4933, -0.2102, -0.4481],
        [-0.4415, -0.4487,  0.4572,  0.4302],
        [ 0.0227, -0.4728, -0.3811, -0.3411]], requires_grad=True)
Parameter containing:
tensor([ 0.5813,  0.9970, -0.2117,  1.1154, -0.3157, -0.2295,  0.2351,  0.0956,
         0.7689, -0.1493,  0.1543, -0.0528, -0.6209,  0.0146, -0.4325,  0.1156],
       requires_grad=True)
Parameter c